In [1]:
import logging
import os
import asyncio

from ogmyrag.report_retrieval.report_retrieval import ReportRetrievalManager
from ogmyrag.report_retrieval.retrieval_embedder import RetrievalEmbedder
from ogmyrag.report_retrieval.retrieval_extractor import RetrievalExtractor
from ogmyrag.report_retrieval.retrieval_storage import RetrievalAsyncStorageManager
from ogmyrag.report_scraper.models import ReportType
from ogmyrag.my_logging import configure_logger

from dotenv import load_dotenv

retrieval_logger = configure_logger(name='retrieval',log_level=logging.INFO, log_file='logs/retrieval.log')
retrieval_logger.info("\n" + "=" * 80)

load_dotenv(override=True)

mongo_db_uri = os.getenv("MONGO_DB_URI_JJ","")
pinecone_api_key = os.getenv("PINECONE_API_KEY_JJ","")
genai_api_key = os.getenv("GENAI_API_KEY_JJ","")
openai_api_key = os.getenv("OPENAI_API_KEY", "")

INDEX_NAME = "markdown-summary-index"
EMBED_MODEL = "text-embedding-3-small"
DIMENSION = 1536
GENAI_MODEL = "gemini-2.5-pro"
OPENAI_MODEL = "gpt-4.1-nano-2025-04-14"



2025-08-03 11:35:55,109 - retrieval - INFO - 


In [2]:
db_name = "FYP"
storage = RetrievalAsyncStorageManager(mongo_uri=mongo_db_uri, db_name=db_name)

embedder = RetrievalEmbedder(
    openai_api_key = openai_api_key,
    pinecone_api_key = pinecone_api_key,
    embed_model = EMBED_MODEL,
    index_name = INDEX_NAME,
    dimension = DIMENSION
)

extractor = RetrievalExtractor(
    openai_api_key = openai_api_key,
    pinecone_api_key = pinecone_api_key,
    embed_model = EMBED_MODEL,
    index_name = INDEX_NAME
)

manager = ReportRetrievalManager(
    storage = storage,
    embedder = embedder,
    extractor = extractor,
    genai_model = GENAI_MODEL,
    genai_api_key = genai_api_key,
    openai_api_key = openai_api_key,
    dry_run = False
)

2025-08-03 11:35:56,726 - retrieval - INFO - Connected to MongoDB database: FYP


## Process Financial Reports (PDF)

In [3]:
manager.parse_store(
    company = "FARM_FRESH_BERHAD",
    report_type = ReportType.ANNUAL,
    year = 2024
)

2025-08-03 11:36:04,976 - retrieval - WARNING - No summary metadata for FARM_FRESH_BERHAD_ANNUAL_2024_summary.md
2025-08-03 11:36:04,996 - retrieval - INFO -      Uploading Farm_Fresh_Berhad_-_Integrated_Annual_Report_2024_(Part_1).pdf ...
2025-08-03 11:36:10,030 - retrieval - INFO -      Uploading Farm_Fresh_Berhad_-_Integrated_Annual_Report_2024_(Part_2).pdf ...
2025-08-03 11:36:14,246 - retrieval - INFO -      Uploading Farm_Fresh_Berhad_-_Integrated_Annual_Report_2024_(Part_3).pdf ...
2025-08-03 11:36:18,339 - retrieval - INFO -      Uploading Farm_Fresh_Berhad_-_Integrated_Annual_Report_2024_(Part_4).pdf ...
2025-08-03 11:36:21,483 - retrieval - INFO -      Uploading Farm_Fresh_Berhad_-_CG_Report_2024.pdf ...
2025-08-03 11:36:25,153 - retrieval - INFO - Uploaded 5 PDFs
2025-08-03 11:36:25,154 - retrieval - INFO - Generating...
2025-08-03 11:38:35,913 - retrieval - INFO - Finished processing.
2025-08-03 11:38:35,917 - retrieval - INFO -      Prompt Tokens: 90485
2025-08-03 11:38:35

## Simple User Query

In [4]:
manager.answer_query(
    company = "FARM_FRESH_BERHAD",
    query = "What is the mission and vision?",
    top_k = 5,
    chat_model = OPENAI_MODEL
)

2025-08-03 11:40:56,916 - retrieval - INFO - Embedding query: 'What is the mission and vision?'
2025-08-03 11:40:58,284 - retrieval - INFO - Retrieved chunks: ['**Vision & mission statement:**', '*   **Our Promise (Vision):** "TO BE A SUSTAINABLE AND HONEST FOOD COMPANY THAT HAS AN INGRAINED CULTURE OF PLACING THE WELL-BEING OF CONSUMERS FIRST, CULMINATING IN STRONG BRAND LOVE." (pg 12)', '*   **Our Action Plan (Mission):** Farm Fresh achieves its brand promise by:', "**Business strategy and outlook:** The Group's strategy is centered on three core pillars: Expansion, Innovation, and Reputation. (pg 62-63)", '## Leadership & Governance']
2025-08-03 11:40:59,134 - retrieval - INFO - Response generated: The vision of the company is to be a sustainable and honest food company that has an ingrained culture of placing the well-being of consumers first, culminating in strong brand love.

The mission focuses on achieving this promise by implementing specific actions, although the detailed act

"The vision of the company is to be a sustainable and honest food company that has an ingrained culture of placing the well-being of consumers first, culminating in strong brand love.\n\nThe mission focuses on achieving this promise by implementing specific actions, although the detailed action plan is outlined in the company's strategy document."